In [68]:
from gurobipy import *
import numpy as np
import math as mt

In [69]:
nb_var = 12
nb_cont = 7

In [70]:
# matrice des contraintes
contraintes = [
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [-1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, -1, 0, -1, 0, 0, 1, 1, -1, 0, 0, 0],
        [0, 0, -1, 0, -1, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1],
]
assert len(contraintes) == 7
assert len(contraintes[0]) == 12

contraintes = np.array(contraintes)
print(contraintes)

[[ 1  1  1  0  0  0  0  0  0  0  0  0]
 [-1  0  0  1  1  1  0  0  0  0  0  0]
 [ 0 -1  0 -1  0  0  1  1 -1  0  0  0]
 [ 0  0 -1  0 -1  0  0  0  1  1  0  0]
 [ 0  0  0  0  0 -1 -1  0  0  0  1  0]
 [ 0  0  0  0  0  0  0 -1  0 -1  0  1]
 [ 0  0  0  0  0  0  0  0  0  0 -1 -1]]


In [71]:
# second membres
# Second membre
S =[1]
S+=[0 for i in range(nb_cont-2)]
S+=[-1]
print(S)

[1, 0, 0, 0, 0, 0, -1]


In [72]:
# Coefficient de la fonction objectif
coef_objectif = [5, 10, 2, 4, 1, 4, 3, 1, 1, 3, 1, 1]
assert len(coef_objectif) == 12

In [73]:
# Creer les models de la fonction objectif
m = Model("1.4-scenario1")

In [74]:
# declaration des variables
vars = ['ab', 'ac', 'ad', 'bc', 'bd', 'be', 'ce', 'cf', 'dc', 'df', 'eg', 'fg']
assert len(vars) == 12

x = []
for node in vars:
    x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="x_%s" % node))


In [75]:
# maj du modele pour integrer les nouvelles variables
m.update()
print(x)

[<gurobi.Var x_ab>, <gurobi.Var x_ac>, <gurobi.Var x_ad>, <gurobi.Var x_bc>, <gurobi.Var x_bd>, <gurobi.Var x_be>, <gurobi.Var x_ce>, <gurobi.Var x_cf>, <gurobi.Var x_dc>, <gurobi.Var x_df>, <gurobi.Var x_eg>, <gurobi.Var x_fg>]


In [76]:
# definition de l'objectif
obj = LinExpr();
obj =0
for j in range(nb_var):
    obj += coef_objectif[j] * x[j]

m.setObjective(obj,GRB.MINIMIZE)

In [77]:
# Definition des contraintes
# Definition des contraintes
lignes = range(nb_cont)
colonnes = range(nb_var)
for i in lignes:
    m.addConstr(quicksum(contraintes[i][j]*x[j] for j in colonnes) == S[i], "Contrainte%d" % i)

In [78]:
# Resolution
m.optimize()


print("")                
print('Solution optimale:')
mvars = m.getVars()
names = m.getAttr('VarName', mvars)
values = m.getAttr('X', mvars)
result = dict(zip(names, values))
for key in result:
    print(key,'=',result[key])
print("")
print('Valeur de la fonction objectif :', m.objVal)

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0x7d9369d7
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 3 rows and 3 columns
Presolve time: 0.01s
Presolved: 4 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+00   2.000000e+00   0.000000e+00      0s
       2    5.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.000000000e+00

Solution optimale:
x_ab = 0.0
x_ac = 0.0
x_ad = 1.0
x_bc = 0.0
x_bd = 0.0
x_be = 0.0
x_ce = 0.0
x_cf = 1.0
x_dc = 1.0
x_df = 0.0
x_eg = 0.0
x